In [ ]:
!nvidia-smi

Sat Jun 25 05:22:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Setup

In [ ]:
!pip install transformers datasets rouge-metric

In [ ]:
import transformers
import torch
import warnings
import time
warnings.filterwarnings("ignore")

4.20.1


In [ ]:
from datasets import load_dataset

dataset = load_dataset('spider')

# dataset["train"]["query"]

Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1034 [00:00<?, ? examples/s]

Dataset spider downloaded and prepared to /root/.cache/huggingface/datasets/spider/spider/1.0.0/4e5143d825a3895451569c8b9b55432b91a4bc2d04d390376c950837f4680daa. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import pandas as pd
df=pd.DataFrame({"db_id":dataset["train"]["db_id"], "query":dataset["train"]["query"],"question":dataset["train"]["question"]})
df.to_csv("Text2Sql.csv")

In [ ]:
df=df.sample(5000)

In [ ]:
df.head()

,db_id,query,question
0,department_management,SELECT count(*) FROM head WHERE age > 56,How many heads of the departments are older th...
1,department_management,"SELECT name , born_state , age FROM head ORD...","List the name, born state and age of the heads..."
2,department_management,"SELECT creation , name , budget_in_billions ...","List the creation year, name and budget of eac..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...",What are the maximum and minimum budget of the...
4,department_management,SELECT avg(num_employees) FROM department WHER...,What is the average number of employees of the...


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M")

Downloading:   0%|          | 0.00/560 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/357 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

In [ ]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=model.to(device)

In [ ]:
device

device(type='cuda')

# Raw model Inference Examples

#Please See

In [ ]:
'''
Parameters---
          max_length= Maximum output length (words)
          top_p=cumululitive probability distribution,
          top_k=sample first top k words [having higher probability]
          temperature=randomness factor [0 to 2] Higher the temeperature more random the generated text will be
          num_return_sequences=  Number of return sequence (top n)

'''

'\nParameters---\n          max_length= Maximum output length (words)\n          top_p=cumululitive probability distribution,\n          top_k=sample first top k words [having higher probability]\n          temperature=randomness factor [0 to 2] Higher the temeperature more random the generated text will be \n          num_return_sequences=  Number of return sequence (top n)\n\n'

In [ ]:
input_sentence="Hello I am Andrew.I am a Computer science student"

In [ ]:
a=time.time()
seq=tokenizer(input_sentence,return_tensors='pt')['input_ids'].to(device)
outputs=model.generate(seq,
                      max_length=40,
                      do_sample=True,
                      top_p=0.95,
                      #num_beams=5,
                      temperature=0.9,
                      no_repeat_ngram_size=3,
                      num_return_sequences=5
                      ).to(device)
for i,out in enumerate(outputs):
  print(f"Sugestion{i} :{tokenizer.decode(out, skip_special_tokens=True)}")
b=time.time()

print("\n",b-a)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sugestion0 :Hello I am Andrew.I am a Computer science student and have a Ph.D. degree in Computer Info. Systems.<|sep|>SELECT DISTINCT T1.engine_
Sugestion1 :Hello I am Andrew.I am a Computer science student who has a Ph.D. degree in the year 2002.<|sep|>SELECT DISTINCT T1.date_of
Sugestion2 :Hello I am Andrew.I am a Computer science student who has a Ph.D. degree in the year 2003.<|sep|>SELECT DISTINCT crs_code FROM Research
Sugestion3 :Hello I am Andrew.I am a Computer science student who is playing the role of goalie.<|sep|>SELECT T1.Andy FROM goalie AS T1 JOIN student AS T2
Sugestion4 :Hello I am Andrew.I am a Computer science student who is a friend of Alice for the longest time.<|sep|>SELECT T1.id, T1 FROM PersonFriend AS T

 0.4895782470703125


# Training

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AdamW, get_linear_schedule_with_warmup
torch.manual_seed(42)

import nltk
import numpy as np
import pandas as pd

### Custom Dataset and DataLoader

In [ ]:
source_len=max(df["question"].apply( lambda x:len(x.split())))*2

target_len=max(df["query"].apply( lambda x:len(x.split())))

In [ ]:
source_len

78

In [ ]:
class custom_Dataset():
    def __init__(self, dataframe, tokenizer, source_len,target_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.target_len = target_len
        self.input_text = dataframe["question"].values
        self.label = dataframe["query"].values

    def __len__(self):
        return len(self.input_text)

    def __getitem__(self,index):
        input_text = str(self.input_text[index])
        input_text = " ".join(input_text.split())

        label = str(self.label[index])
        label = " ".join(label.split())

        final_text=input_text+'<|sep|>'+label
        source = self.tokenizer.encode_plus(
            final_text,
            max_length=self.source_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        target = self.tokenizer.encode_plus(
            label,
            max_length=self.target_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )

        source_ids = source["input_ids"].squeeze()
        # target_ids=target["input_ids"].squeeze()
        source_mask = source["attention_mask"].squeeze()

        return (source_ids.to(dtype=torch.long),source_mask.to(dtype=torch.long),)



In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
len(tokenizer)

50257

In [ ]:
dataset = custom_Dataset(df, tokenizer, source_len=source_len,target_len=target_len)
train_dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_dataloader=DataLoader(train_dataset,batch_size=16,shuffle=True)
val_dataloader=DataLoader(val_dataset,batch_size=1,shuffle=True)


In [ ]:
x,y=next(iter(train_dataloader))
x=x.to(device)
y=y.to(device)

In [ ]:
x.shape

torch.Size([16, 78])

In [ ]:
X

In [ ]:
y.shape

torch.Size([16, 78])

In [ ]:
out=model(x,labels=x,attention_mask = y)

In [ ]:
torch.argmax(out["logits"],2).to(dtype=torch.float)

In [ ]:
out["logits"].permute(0,2,1).shape

torch.Size([16, 50257, 32])

In [ ]:
import torch.nn as nn
criterion=nn.CrossEntropyLoss()

In [ ]:
# criterion(out["logits"].permute(0,2,1),z)

In [ ]:
### Cross entropy loss shape pronlem

# https://www.pythonfixing.com/2021/10/fixed-pytorch-nncrossentropyloss-giving.html

# https://discuss.pytorch.org/t/valueerror-expected-target-size-32-7-got-torch-size-32/42409

In [ ]:
import gc
gc.collect()
del x,y

In [ ]:
gc.collect()

44

In [ ]:
learning_rate = 1e-4
warmup_steps = 1e2
epsilon = 1e-8

optimizer = AdamW(model.parameters(),
                  lr = learning_rate,
                  eps = epsilon
                )

In [ ]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=model.to(device)

In [ ]:
[1,2,3].mean()

In [ ]:
def evaluate(val_dataloader,model):
    accuracy=[]
    model.eval()
    # with torch.no_grad():
    for batch in val_dataloader:
        input_ids = batch[0].to(device)
        outputs=model.generate(input_ids,
                          max_length=40,
                          do_sample=True,
                          top_p=0.95,
                          #num_beams=5,
                          temperature=0.7,
                          no_repeat_ngram_size=3,
                          num_return_sequences=1
                          ).to(device)
        inp=tokenizer.decode(input_ids[0], skip_special_tokens=False)
        out=tokenizer.decode(outputs[0], skip_special_tokens=True)
        # print(inp)
        # print(out)
        idx=out.find("<|sep|>")+7
        out=out[idx:]
        scores = rouge.get_scores(out, inp)
        acc=scores[0]["rouge-2"]["f"]
        accuracy.append(acc)
    accuracy=sum(accuracy)/len(accuracy)
    return accuracy

evaluate(val_dataloader,model)

In [ ]:
# tokenizer.decode(torch.tensor([ 2061,   389,   262,  1180, 12834,  5610,   287,   262,  6831,    30,
#            27,    91,   325,    79,    91,    29, 46506,   360,  8808,  1268,
#          4177,  8875, 16034, 43953, 50256, 50256, 50256, 50256, 50256, 50256,
#         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
#         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
#         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
#         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
#         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256]),special_tokens=False)

In [ ]:
epochs= 10

for epoch_i in range(epochs):
    print("")
    print('Training...')

    total_train_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):
        model.zero_grad()
        input_ids = batch[0].to(device)
        # target_ids=batch[1].to(device)
        attn_ids=batch[1].to(device)
        model.zero_grad()

        outputs = model(input_ids,
                        labels=input_ids,
                        attention_mask = attn_ids
                        )
        # print(outputs)
        loss = outputs[0]
        # loss=criterion(outputs["logits"].permute(0,2,1),target_ids)
        batch_loss = loss
        total_train_loss += batch_loss.item()
        loss.backward()
        optimizer.step()
        del batch
        if step%200==0:
            print(f"Step:{step}=======Losss:{loss}")
    print('======== Epoch {:} / {:} ======== LOSS... {}'.format(epoch_i + 1, epochs,total_train_loss/len(train_dataloader)))



Training...
Step:0=======Losss:4.591061592102051
Step:200=======Losss:1.2315764427185059
======== Epoch 1 / 10 ======== LOSS... 1.635881196195706

Training...
Step:0=======Losss:1.287850260734558
Step:200=======Losss:0.977429986000061
======== Epoch 2 / 10 ======== LOSS... 1.1196162969159622

Training...
Step:0=======Losss:0.9737753868103027
Step:200=======Losss:0.9129559993743896
======== Epoch 3 / 10 ======== LOSS... 0.871231882716901

Training...
Step:0=======Losss:0.6815928816795349
Step:200=======Losss:0.6060318946838379
======== Epoch 4 / 10 ======== LOSS... 0.6919562590007965

Training...
Step:0=======Losss:0.5223892331123352
Step:200=======Losss:0.5060715675354004
======== Epoch 5 / 10 ======== LOSS... 0.558649618499957

Training...
Step:0=======Losss:0.45613160729408264
Step:200=======Losss:0.47165119647979736
======== Epoch 6 / 10 ======== LOSS... 0.46264086620876205

Training...
Step:0=======Losss:0.4107467234134674
Step:200=======Losss:0.44260066747665405
======== Epoch 7 

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

model=torch.load("/content/drive/MyDrive/text2sql/Gpt_neo_Epoch_10_Loss_031_data_5000.pth")
model=model.to(device)

In [ ]:
model=model.to(device)

In [ ]:
def predict_query(input_sentence,temp=0.7):
    a=time.time()
    seq=tokenizer(input_sentence,return_tensors='pt')['input_ids'].to(device)
    outputs=model.generate(seq,
                          max_length=40,
                          do_sample=True,
                          top_p=0.95,
                          #num_beams=5,
                          temperature=temp,
                          no_repeat_ngram_size=3,
                          num_return_sequences=5
                          ).to(device)
    for i,out in enumerate(outputs):
      out=tokenizer.decode(out, skip_special_tokens=True)
      idx=out.find("<|sep|>")+7
      print(f"Sugestion{i} :{out[idx:]}")
    b=time.time()
    print("\n",b-a)


In [ ]:
input_sentence="Find total Number of airplanes landing on USA airport"
torch.manual_seed(0)
predict_query(input_sentence)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sugestion0 :SELECT count(*) FROM Flight WHERE country = "USA"
Sugestion1 :SELECT count(*) FROM Flight WHERE origin = 'USA'
Sugestion2 :SELECT sum(total) FROM Flight WHERE country = 'USA'
Sugestion3 :SELECT count(*) FROM Flight WHERE country = "USA"
Sugestion4 :SELECT sum(total) FROM Flight WHERE country = 'USA'

 0.43268918991088867


In [ ]:
input_sentence="List all students scoring greater than 90 percent from student table"
predict_query(input_sentence)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sugestion0 :SELECT Stu_ID FROM STUDENT WHERE Credits > 90 GROUP BY Stu1
Sugestion1 :SELECT StuID FROM STUDENT WHERE grade > 90 GROUP BY Stu
Sugestion2 :SELECT DISTINCT stu_code FROM player WHERE Stu_ID = 90
Sugestion3 :SELECT StuID FROM STUDENT WHERE enr = "cientious" AND scored > 90
Sugestion4 :SELECT StuID FROM Student WHERE age > 90 GROUP BY StuCAPT1

 0.40462636947631836


In [ ]:
input_sentence="Select all students who got placed in company 'AMAZON'"
predict_query(input_sentence)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sugestion0 :SELECT DISTINCT company FROM company WHERE id NOT IN
Sugestion1 :SELECT DISTINCT T1.pName FROM student AS
Sugestion2 :SELECT T1.student_name FROM student_company AS T
Sugestion3 :SELECT DISTINCT T1.name FROM student AS T1 JOIN
Sugestion4 :SELECT DISTINCT T1.student_name FROM

 0.7771613597869873


In [ ]:
input_sentence="Calculate Average Salary of Employee"
predict_query(input_sentence)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sugestion0 :SELECT avg(T1.salary), T2.name FROM salary_billions
Sugestion1 :SELECT avg(T1.salary), T2.name FROM salary
Sugestion2 :SELECT avg(salary), salary FROM employees WHERE salary BETWEEN 7000 AND 12
Sugestion3 :SELECT avg(salary) FROM Employee WHERE salary ='m'
Sugestion4 :SELECT avg(salary) FROM Employee WHERE Age > 40

 0.4919166564941406


In [ ]:
break

SyntaxError: ignored

In [ ]:
torch.save(model,"Gpt_neo_Epoch_10_Loss_031_data_5000.pth")

In [ ]:
import shutil
shutil.move("/content/Gpt_neo_Epoch_10_Loss_031_data_5000.pth","/content/drive/MyDrive/text2sql")

# Evaluating

In [ ]:
!pip install rouge


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from rouge import Rouge

hypothesis='SELECT count(*) FROM Boeing WHERE country = "USA" AND flag = " visibility"'

reference='SELECT count(*) FROM Flight WHERE country = "USA"'
rouge = Rouge()
scores = rouge.get_scores(hypothesis, reference)
scores

[{'rouge-1': {'f': 0.6999999952000001, 'p': 0.5833333333333334, 'r': 0.875},
  'rouge-2': {'f': 0.5263157848199447,
   'p': 0.4166666666666667,
   'r': 0.7142857142857143},
  'rouge-l': {'f': 0.6999999952000001, 'p': 0.5833333333333334, 'r': 0.875}}]

In [ ]:
scores[0]["rouge-2"]["f"]

0.5263157848199447

In [ ]:
model.eval()

for step, batch in enumerate(train_dataloader):
        r=[]
        input_ids = batch[0].to(device)
        attn_ids=batch[1].to(device)

        p=predict_query(input_sentence,temp=0.7)
        # print(p[0])
        score=rouge.get_scores(tokenizer.decode(input_ids, skip_special_tokens=True), p[0])
        r.append(score["rouge-2"]["f"])
        # print(outputs)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sugestion0 :SELECT T2.name FROM user
Sugestion1 :SELECT T2.id, T1.name FROM student AS T1 JOIN credit
Sugestion2 :SELECT T1.id, T2.title
Sugestion3 :SELECT T2.name FROM Person AS T1 JOIN
Sugestion4 :SELECT T2.name FROM student AS T1 JOIN instructor AS T2 ON T1.id = T

 0.7595479488372803


TypeError: ignored